In [135]:
# Setup
import pandas as pd
from pathlib import Path
data_path = Path('./data/food/')

# 'country_lookup' from '0A - Food System POPULATION inputs 7-5-21.xlsx' workbook:
#  List of countries included in FAO analysis, with various categories used throughout the Food Systems model
countries = pd.read_csv(Path.joinpath(data_path,'countries.txt'), index_col='Country').convert_dtypes()
#old_countries = (countries['CountryExists'] == 0)
#old_country_set = set(c.casefold() for c in countries[old_countries].index.to_list())

# 'LCA range' from '0B - Food System LCA inputs 7-5-21.xslx' workbook.
# Farm gate LCA impacts by FAO commodity, with max/min/avg values by study. (g CO2e/kcal food)
LCA_emissions_by_product = pd.read_csv(Path.joinpath(data_path,'LCA_emissions_by_product.txt'), index_col='FAO commodity').convert_dtypes()
LCA_emissions_by_product.drop('Sort code', axis='columns', inplace=True)

# 'REF_supply' from '0C - Food System FOOD inputs 75 6-19-21.xslx' workbook.
# REF farm gate food supply forecast by country and FAO commodity [kcal/(cap-day)] 
REF_supply = pd.read_csv(Path.joinpath(data_path,'REF_supply.txt'), index_col=['Country', 'FAO commodity']).convert_dtypes()
# idx_countries = set(REF_supply.index.get_level_values(level=0).to_list())
# drop_countries = []
# for idx_country in idx_countries:
#     if idx_country.casefold() in old_country_set:
#         drop_countries.append(idx_country)

cols = REF_supply.columns.to_list()
years = [col for col in cols if len(col)==4 and col[:1]=='2']
years_int = [int(y) for y in years]

# 'REF_waste' table from REF_waste tab in '0C - Food System FOOD inputs 75 6-19-21.xslx' workbook.
# REF food waste forecast by country and FAO commodity [kcal/(cap-day)]
REF_waste = pd.read_csv(Path.joinpath(data_path,'REF_waste.txt'), index_col=['Country', 'FAO commodity']).convert_dtypes()

# 'waste_forecast' table from waste_forecast tab in '0C - Food System FOOD inputs 75 6-19-21.xslx' workbook.
# Mass percentages of food losses and waste (in percentage of what enters each step) (Source: FAO 2011, Global Food Losses.)
waste_forecast = pd.read_csv(Path.joinpath(data_path,'waste_forecast.txt'), index_col=['Waste category', 'Waste region']).convert_dtypes()

# Taken from the 'food_lookup' table on the 'food_lookup' tab in the '0C - Food System FOOD Inputs 75 6-19-21.xlsx' spreadsheet.
food_lookup  = pd.read_csv(Path.joinpath(data_path,'food_lookup.txt'), index_col=['FAO commodity']).convert_dtypes()

# From food_lookup tab in "1 - Food System CORE 75 6-18-21.xlsx"
# FAO commodity, Waste category, Diet cateory, Demand category, kcal/g, Yield category:
# List of foods included in the FAO commodity list, with various categories used throughout the Food Systems model.
food_waste_lookup = pd.read_csv(Path.joinpath(data_path,'food_waste_lookup.txt'), index_col=['FAO commodity']).convert_dtypes()

# plant_rich_diet taken from the 'plant-rich_diet' tab on the '0C - Food System FOOD Inputs 75 6-19-21.xlsx' spreadsheet.
# There is no commentary in the spreadsheet as to how the data is derived. This tab also contains the following erroneous country names: "C√É¬¥te d'Ivoire", "Czechia"
plant_rich_diet = pd.read_csv(Path.joinpath(data_path,'plant_rich_diet.txt'), index_col=['Country', 'FAO commodity']).convert_dtypes() # fbs = Food Balance Sheet

# UN Population estimates from World Population Prospects, High Population Scenario, 2014-2100
# Taken from '0A - Food System POPULATION inputs 7-5-21.xlsx', pop_REF1 tab.
population_high = pd.read_csv(Path.joinpath(data_path,'population_high.txt'), index_col='Country').convert_dtypes()

# UN Population estimates from World Population Prospects, Medium Population Scenario (reflecting "family planning" scenario intervention), 2014-2100
# Taken from '0A - Food System POPULATION inputs 7-5-21.xlsx', pop_REF2 tab.
population_medium = pd.read_csv(Path.joinpath(data_path,'population_medium.txt'), index_col='Country').convert_dtypes()

parameters = {
    'localization_emisssion_reduction_percentage': 0.05,
    'total_food_waste_reduction': 0.75,
    'diet_start_year': 2019,
    'diet_end_year': 2050,
    'waste_start_year': 2019,
    'waste_end_year': 2050,
    'plant_rich_diet_adoption': 0.75,
    'plant_rich_diet_kcal_per_day': 2300
    }


In [136]:
# Matches waste_forecast table on "Waste_forecast" tab in "1 - Food System CORE 75 6-18-21.xlsx"
waste_forecast['Post-farm gate loss'] =  1-(1-waste_forecast['Postharvest handling and storage'])*(1-waste_forecast['Processing'])*(1-waste_forecast['Packaging']) * \
                                        (1-waste_forecast['Distribution'])*(1-waste_forecast['Consumption'])

# @IF(AND(NUMBERVALUE(waste_forecast[[#Headers],[2014]])>=parameters!$E$6,NUMBERVALUE(waste_forecast[[#Headers],[2014]])<=parameters!$F$6),
#        @waste_forecast[@[Post-farm gate loss]:[Post-farm gate loss]]*(1-parameters!$A$7*(NUMBERVALUE(waste_forecast[[#Headers],[2014]])-parameters!$E$6+1)/(parameters!$F$6-parameters!$E$6+1)),
#       IF(NUMBERVALUE(waste_forecast[[#Headers],[2014]])>parameters!$F$6,@waste_forecast[@[Post-farm gate loss]:[Post-farm gate loss]]*(1-parameters!$A$7),waste_forecast[@[Post-farm gate loss]:[Post-farm gate loss]]))

start_idx = years_int.index(parameters['waste_start_year'])
end_idx = years_int.index(parameters['waste_end_year'])

for y in years_int[:start_idx]:
    waste_forecast[str(y)] = waste_forecast['Post-farm gate loss']

for y in years_int[start_idx:end_idx]:
    waste_forecast[str(y)] = waste_forecast['Post-farm gate loss'] * (1-parameters['total_food_waste_reduction'] * (y - parameters['waste_start_year'] + 1 )/ (parameters['waste_end_year'] - parameters['waste_start_year'] + 1))

for y in years_int[end_idx:]:
    waste_forecast[str(y)] = waste_forecast['Post-farm gate loss'] * (1-parameters['total_food_waste_reduction'])



In [157]:
# Build DIET_supply and DIET_plant_supply to match the two tables on the DIET_supply tab from "1 - Food System CORE 75 6-18-21.xlsx".

# DIET_supply: Per-capita DIET-only total food supply forecast, 2014-2100 [kcal/(cap-day)]
# DIET_plant_supply: DIET food supply forecast *plant-rich diet only*, 2014-2100 [kcal/(cap-day)]

plantrich_demand_2000 = plant_rich_diet.join(other=food_lookup[['Diet category','Waste category']], how='inner').join(countries[['Diet region','Waste region']])[['GrandTotal', 'Diet category', 'Waste category', 'Diet region', 'Waste region']]
plantrich_demand = plantrich_demand_2000.copy()
plantrich_demand['GrandTotal'] = plantrich_demand_2000['GrandTotal'] * parameters['plant_rich_diet_kcal_per_day'] / 2300

diet_factors = []
start_yr = parameters['diet_start_year']
end_yr = parameters['diet_end_year']
adoption = parameters['plant_rich_diet_adoption']
denominator = end_yr - start_yr + 1

for y in range(years_int[0],start_yr):
    diet_factors.insert(0,0.0)

for y in range(start_yr, end_yr+1):
    val =  adoption * (y - start_yr +1) / denominator
    diet_factors.append(val)

for y in range(end_yr, years_int[-1]):
    diet_factors.append(adoption)

REF_demand = REF_supply[years] - REF_waste[years]
DIET_demand = REF_demand.join(food_lookup[['Waste category','Diet category']]).join(plantrich_demand.GrandTotal, how='left')
DIET_plant_supply = plantrich_demand.copy()
idx = DIET_plant_supply.index
DIET_plant_supply = pd.merge(DIET_plant_supply, waste_forecast['Post-farm gate loss'], how='left', on=['Waste category','Waste region'])
DIET_plant_supply.set_index(idx, inplace=True)

DIET_demand[years] = DIET_demand[years].mul([1-d for d in diet_factors])

wk_DIET_demand = pd.DataFrame(DIET_demand.GrandTotal.fillna(0.0))
grand_total_factors = pd.DataFrame.from_dict(data={'GrandTotal': diet_factors}, orient='index', columns=years)
wk_DIET_demand = wk_DIET_demand.dot(grand_total_factors)

wk_DIET_plant_supply = pd.DataFrame(DIET_plant_supply.GrandTotal/(1-DIET_plant_supply['Post-farm gate loss']),columns=['GrandTotal'])
wk_DIET_plant_supply = wk_DIET_plant_supply.dot(grand_total_factors)
    
# DIET_demand[@2014]*VLOOKUP(@DIET_waste[@[Waste lookup]:[Waste lookup]],waste_forecast[#All],11,0)/(1-VLOOKUP(@DIET_waste[@[Waste lookup]:[Waste lookup]],waste_forecast[#All],11,0))
# DIET_waste = DIET_demand.merge(waste_forecast['Post-farm gate loss'])
DIET_waste = DIET_demand.join(countries['Waste region'])
idx = DIET_waste.index
DIET_waste = pd.merge(DIET_waste, waste_forecast['Post-farm gate loss'], on=['Waste region', 'Waste category'], how='left')
DIET_waste.set_index(idx,inplace=True)

pfg = pd.to_numeric(DIET_waste['Post-farm gate loss'].map(lambda x: x/(1-x))).to_frame().reindex(labels=years, axis='columns', method='backfill')

DIET_waste[years] = DIET_waste[years].mul(pfg)

DIET_supply = DIET_demand[years] + DIET_waste[years]
# DIET_waste, DIET_demand and DIET_supply are now correct.


In [4]:
# Per-capita PDS total food supply forecast, 2014-2100 [kcal/(cap-day)]
# PDS_supply from "1 - Food System CORE 75 6-18-21.xlsx".

# PDS_demand:

PDS_demand = REF_demand.copy()
for i, y in enumerate(years):
    fact = diet_factors[i]
    PDS_demand[y] = PDS_demand[y] * (1-fact)
    PDS_demand[y] = PDS_demand[y].add(plantrich_demand['GrandTotal'] * fact, fill_value=0.0)

# PDS_demand now complete

# next for PDS_waste

PDS_waste = PDS_demand.join(food_lookup['Waste category']).join(countries['Waste region'])
w_suffix, f_suffix = ('_waste','_forecast')
idx = PDS_waste.index
PDS_waste = pd.merge(PDS_waste, waste_forecast.reset_index(), on=['Waste category', 'Waste region'], how='left', suffixes=[w_suffix, f_suffix])
PDS_waste.set_index(idx, inplace=True)

for y in years:
    w_idx = y + w_suffix
    f_idx = y + f_suffix
    PDS_waste[y] = PDS_waste[w_idx] * PDS_waste[f_idx] / (1-PDS_waste[f_idx])
    
PDS_waste = PDS_waste[years]

# PDS_waste now complete
PDS_supply = PDS_waste + PDS_demand


In [5]:
#plantrich_demand#.loc[(slice(None),'Aquatic Animals, Others'),:]
#REF_demand.loc[(slice(None),'Aquatic Animals, Others'),:]
#PDS_demand.loc[(slice(None),'Aquatic Animals, Others'),:]
#PDS_waste.loc[(slice(None),'Aquatic Animals, Others'),:]
#PDS_supply.loc[(slice(None),'Aquatic Animals, Others'),:]

In [6]:
# PDS food supply forecast *plant-rich diet only*, 2014-2100 [kcal/(cap-day)]
# PDS_plant_supply table on the PDS_supply tab in the "1 - Food System CORE 75 6-18-21.xlsx" workbook.
# formula e.g. PDS_plant_supply = plantrich_demand[@2017]*PDS_demand!G$2/(1-INDEX(waste_forecast[#All],MATCH(@PDS_plant_supply[@[Waste lookup]:[Waste lookup]],waste_forecast[[#All],[Waste lookup]:[Waste lookup]],0),MATCH(PDS_plant_supply[[#Headers],[2017]],waste_forecast[#Headers],0)))

# pds_supply_plant_rich = pd.read_csv(r'/mnt/c/Users/neilm/Documents/Drawdown/Excel Files/food_system/data/PDS_food_supply_plant_rich_diet.txt', index_col=['Country', 'FAO commodity'])

# PDS_plant_supply = pds_supply_plant_rich comes from plantrich_demand, which comes from plant_rich_diet, which is read in from a file, which represents the controversial table with GrandTotal.

#PDS_plant_supply
PDS_plant_supply = PDS_demand.join(food_lookup['Waste category']).join(countries['Waste region'])

d_suffix, f_suffix = ('_demand','_forecast')
idx = PDS_plant_supply.index
PDS_plant_supply = pd.merge(PDS_plant_supply, waste_forecast.reset_index(), on=['Waste category', 'Waste region'], how='left', suffixes=[d_suffix, f_suffix])
PDS_plant_supply.set_index(idx, inplace=True)

for i, y in enumerate(years):
    fact = diet_factors[i]
    f_idx = y + f_suffix
    PDS_plant_supply[y] = plantrich_demand['GrandTotal'] * fact / (1-PDS_plant_supply[f_idx])

PDS_plant_supply = PDS_plant_supply[years]
# PDS_plant_supply now matches PDS_plant_supply table on the PDS_supply tab in the "1 - Food System CORE 75 6-18-21.xlsx" workbook.

In [7]:
# Per-capita WASTE-only total food supply forecast, 2014-2100 [kcal/(cap-day)]
# WASTE_supply from "1 - Food System CORE 75 6-18-21.xlsx".

# formula e.g. =SUM(WASTE_waste[@2017],WASTE_demand[@2017])
# waste_supply = pd.read_csv(r'/mnt/c/Users/neilm/Documents/Drawdown/Excel Files/food_system/data/WASTE_supply.txt', index_col=['Country', 'FAO commodity'])

# Need to calculate WASTE_waste and WASTE_demand # Note: WASTE_demand = REF_demand in the spreadsheet
# To calculate WASTE_waste:
# WASTE_demand[@2014]*INDEX(waste_forecast[#All],MATCH(@WASTE_waste[@[Waste lookup]:[Waste lookup]],waste_forecast[[#All],[Waste lookup]:[Waste lookup]],0),MATCH(WASTE_waste[[#Headers],[2014]],waste_forecast[#Headers],0))/(1-INDEX(waste_forecast[#All],MATCH(@WASTE_waste[@[Waste lookup]:[Waste lookup]],waste_forecast[[#All],[Waste lookup]:[Waste lookup]],0),MATCH(WASTE_waste[[#Headers],[2014]],waste_forecast[#Headers],0)))


WASTE_waste = REF_demand.join(food_lookup['Waste category']).join(countries['Waste region'])
w_suffix, f_suffix = ('_waste','_forecast')
idx = WASTE_waste.index
WASTE_waste = pd.merge(WASTE_waste, waste_forecast.reset_index(), on=['Waste category', 'Waste region'], how='left', suffixes=[w_suffix, f_suffix])
WASTE_waste.set_index(idx, inplace=True)

old_cols = []
for y in years:
    w_idx = y + w_suffix
    f_idx = y + f_suffix
    WASTE_waste[y] = WASTE_waste[w_idx] * WASTE_waste[f_idx] / (1-WASTE_waste[f_idx])
    old_cols.append(w_idx)
    old_cols.append(f_idx)
    
#WASTE_waste.drop(labels=old_cols, axis='columns', inplace=True)
#WASTE_waste = WASTE_waste.set_index(['Waste category', 'Waste region'])
WASTE_waste = WASTE_waste[years]

WASTE_supply = WASTE_waste + REF_demand
#food_waste_lookup

In [8]:
# REF_emissions
REF_emissions_work = REF_supply.copy()
REF_emissions_work.drop(labels=drop_countries, level=0, axis='rows', inplace=True)

REF_emissions_work = REF_emissions_work.loc[:,years].multiply(population_high)
REF1_emissions = LCA_emissions_by_product.join(other=REF_emissions_work)
avg = REF1_emissions['Avg']

for y in years:
    REF1_emissions[y] *= avg * 365 / 10**6

REF2_emissions_work = REF_supply.copy()
REF2_emissions_work.drop(labels=drop_countries, level=0, axis='rows', inplace=True)

REF2_emissions_work = REF2_emissions_work.loc[:,years].multiply(population_medium)
REF2_emissions = LCA_emissions_by_product.join(other=REF2_emissions_work)
avg = REF2_emissions['Avg']

for y in years:
    REF2_emissions[y] *= avg * 365 / 10**6


In [9]:
# PDS_emissions

diet_diff = DIET_supply.copy() # old = diet_supply_core
diet_supply_plant_rich_work = DIET_plant_supply.copy() # old = diet_supply_plant_rich

diet_diff.drop(labels=drop_countries, level=0, axis='rows', inplace=True)
#diet_supply_plant_rich_work.drop(labels=drop_countries, level=0, axis='rows', inplace=True)

for y in years:
    diet_diff[y] -= diet_supply_plant_rich_work[y]
 
avg = LCA_emissions_by_product['Avg']
diet_supply_core_work = diet_diff.copy()
diet_supply_core_work2 = DIET_plant_supply.copy() # old = diet_supply_plant_rich
#diet_supply_core_work2.drop(labels=drop_countries, level=0, axis='rows', inplace=True)

for y in years:
    diet_supply_core_work[y] *= avg
    diet_supply_core_work2[y] *= avg
    
    diet_supply_core_work[y] *= population_medium[y]
    diet_supply_core_work2[y] *= population_medium[y]
    if int(y) >= parameters['diet_start_year']:
        diet_supply_core_work2[y] *= (1-parameters['localization_emisssion_reduction_percentage'])

diet_annual_emissions = (diet_supply_core_work + diet_supply_core_work2) * 365/10**6

# calculate waste emissions:
waste_emissions = WASTE_supply.copy()
waste_emissions.drop(labels=drop_countries, level=0, axis='rows', inplace=True)
for y in years:
    waste_emissions[y] *= avg
    waste_emissions[y] *= population_medium[y]
    waste_emissions[y] *= 365/(10**6)

# calculate PDS annual emissions:
pds_diff = PDS_supply.copy() # old: pds_supply_core
pds_diff.drop(labels=drop_countries, level=0, axis='rows', inplace=True)
for y in years:
    pds_diff[y] -= PDS_plant_supply[y] # old = pds_supply_plant_rich[y]
    
pds_supply_core_work = pds_diff.copy()
pds_supply_core_work.drop(labels=drop_countries, level=0, axis='rows', inplace=True)
pds_supply_core_work2 = PDS_plant_supply.copy() # old = pds_supply_plant_rich
pds_supply_core_work2.drop(labels=drop_countries, level=0, axis='rows', inplace=True)

for y in years:
    pds_supply_core_work[y] *= avg
    pds_supply_core_work2[y] *= avg
    
    pds_supply_core_work[y] *= population_medium[y]
    pds_supply_core_work2[y] *= population_medium[y]
    if int(y) >= parameters['diet_start_year']:
        pds_supply_core_work2[y] *= (1-parameters['localization_emisssion_reduction_percentage'])

pds_annual_emissions = (pds_supply_core_work + pds_supply_core_work2) * 365/10**6


In [10]:
# REF_country_CO2
REF1_country_per_capita_emissions = REF1_emissions.loc[:,years].groupby(level='Country').sum()
for y in years:
    REF1_country_per_capita_emissions[y] /= (population_high[y] * 365 / (10**3) )

REF2_country_per_capita_emissions = REF2_emissions.loc[:,years].groupby(level='Country').sum()
for y in years:
     REF2_country_per_capita_emissions[y] /= (population_medium[y] * 365 / (10**3) )


In [11]:
# PDS_country_CO2
pds_diet_per_capita_emissions = diet_annual_emissions.groupby(level='Country').sum()
waste_per_capita_emissions = waste_emissions.groupby(level='Country').sum()
pds_per_capita_emissions = pds_annual_emissions.groupby(level='Country').sum()

for y in years:
    population = (population_medium[y] * 365 / (10**3) )
    pds_diet_per_capita_emissions[y] /= population
    waste_per_capita_emissions[y] /= population
    pds_per_capita_emissions[y] /= population

# Note: [NL, 14 Oct 2021], the following three tables have incorrect formulas in the spreadsheet:
# pds_diet_per_capita_emissions, waste_per_capita_emissions, pds_per_capita_emissions


In [12]:
# REF_commodity_CO2
REF1_commodity_CO2 = REF1_emissions.groupby(level='FAO commodity').sum() # Note - doesn't match with s/sht. Formula hasn't been copied down properly.
REF2_commodity_CO2 = REF2_emissions.groupby(level='FAO commodity').sum() # Note - doesn't match with s/sht. Formula hasn't been copied down properly.

# PDS_commodity_CO2
diet_commodity_CO2 = diet_annual_emissions.groupby(level='FAO commodity').sum() # Note - doesn't match with s/sht. Formula hasn't been copied down properly.
waste_commodity_CO2 = waste_emissions.groupby(level='FAO commodity').sum() # Note - doesn't match with s/sht. Formula hasn't been copied down properly.
pds_commodity_CO2 = pds_annual_emissions.groupby(level='FAO commodity').sum() # Note - doesn't match with s/sht. Formula hasn't been copied down properly.

In [13]:
# Global yield

REF_supply_year_yield_pop_ref1 = REF_supply.copy()
REF_supply_year_yield_pop_ref2 = REF_supply.copy()
diet_supply_year_yield_pop_ref2 = DIET_supply.copy() # old: diet_supply_core
waste_supply_year_yield_pop_ref2 = WASTE_supply.copy() # old: waste_supply
pds_supply_year_yield_pop_ref2 = PDS_supply.copy() # old: pds_supply_core

kcal_per_gram = REF_supply_year_yield_pop_ref1['kcal/g']
REF_supply_year_yield_pop_ref1 = REF_supply_year_yield_pop_ref1.loc[:,years].divide(other=kcal_per_gram, axis='index')
REF_supply_year_yield_pop_ref1 = REF_supply_year_yield_pop_ref1.multiply(other=population_high, fill_value=0.0)
REF_supply_year_yield_pop_ref1 *= 365

REF_supply_year_yield_pop_ref2 = REF_supply_year_yield_pop_ref2.loc[:,years].divide(other=kcal_per_gram, axis='index')
REF_supply_year_yield_pop_ref2 = REF_supply_year_yield_pop_ref2.multiply(other=population_medium, fill_value=0.0)
REF_supply_year_yield_pop_ref2 *= 365

diet_supply_year_yield_pop_ref2 = diet_supply_year_yield_pop_ref2.loc[:,years].divide(other=kcal_per_gram, axis='index')
diet_supply_year_yield_pop_ref2 = diet_supply_year_yield_pop_ref2.multiply(other=population_medium, fill_value=0.0)
diet_supply_year_yield_pop_ref2 *= 365

waste_supply_year_yield_pop_ref2 = waste_supply_year_yield_pop_ref2.loc[:,years].divide(other=kcal_per_gram, axis='index')
waste_supply_year_yield_pop_ref2 = waste_supply_year_yield_pop_ref2.multiply(other=population_medium, fill_value=0.0)
waste_supply_year_yield_pop_ref2 *= 365

pds_supply_year_yield_pop_ref2 = pds_supply_year_yield_pop_ref2.loc[:,years].divide(other=kcal_per_gram, axis='index')
pds_supply_year_yield_pop_ref2 = pds_supply_year_yield_pop_ref2.multiply(other=population_medium, fill_value=0.0)
pds_supply_year_yield_pop_ref2 *= 365


report_dict = {'Grain': {'Cereals', 'Fruits and vegetables', 'Oilseeds and pulses', 'Roots and tubers', 'Other'},
               'Meat': {'Bovine Meat', 'Meat, Other', 'Mutton & Goat Meat', 'Pigmeat', 'Poultry Meat', 'Fats, Animals, Raw', 'Offals, Edible'},
               'Milk and egg': {'Butter, Ghee', 'Cream', 'Eggs', 'Milk - Excluding Butter', 'Whey'},
               'Fish and seafood': {'Aquatic Animals, Others', 'Aquatic Plants', 'Cephalopods', 'Crustaceans', 'Demersal Fish', 'Fish, Body Oil', 'Fish, Liver Oil',
                                    'Freshwater Fish', 'Marine Fish, Other',  'Meat, Aquatic Mammals', 'Molluscs, Other', 'Pelagic Fish'}
               }

REF_supply_year_yield_pop_ref1_yc = REF_supply_year_yield_pop_ref1.join(food_waste_lookup['Yield category']).groupby('Yield category').sum()
REF_supply_year_yield_pop_ref1_yc.insert(loc=0, column='Report Category', value='', allow_duplicates = True)

REF_supply_year_yield_pop_ref2_yc = REF_supply_year_yield_pop_ref2.join(food_waste_lookup['Yield category']).groupby('Yield category').sum()
REF_supply_year_yield_pop_ref2_yc.insert(loc=0, column='Report Category', value='', allow_duplicates = True)

diet_supply_year_yield_pop_ref2_yc = diet_supply_year_yield_pop_ref2.join(food_waste_lookup['Yield category']).groupby('Yield category').sum()
diet_supply_year_yield_pop_ref2_yc.insert(loc=0, column='Report Category', value='', allow_duplicates = True)

waste_supply_year_yield_pop_ref2_yc = waste_supply_year_yield_pop_ref2.join(food_waste_lookup['Yield category']).groupby('Yield category').sum()
waste_supply_year_yield_pop_ref2_yc.insert(loc=0, column='Report Category', value='', allow_duplicates = True)

pds_supply_year_yield_pop_ref2_yc = pds_supply_year_yield_pop_ref2.join(food_waste_lookup['Yield category']).groupby('Yield category').sum()
pds_supply_year_yield_pop_ref2_yc.insert(loc=0, column='Report Category', value='', allow_duplicates = True)

idx_vals = set(REF_supply_year_yield_pop_ref1_yc.index.to_list())

for report_category, yield_categories in report_dict.items():
    REF_supply_year_yield_pop_ref1_yc.loc[yield_categories & idx_vals, 'Report Category'] = report_category
    REF_supply_year_yield_pop_ref2_yc.loc[yield_categories & idx_vals, 'Report Category'] = report_category
    diet_supply_year_yield_pop_ref2_yc.loc[yield_categories & idx_vals, 'Report Category'] = report_category
    waste_supply_year_yield_pop_ref2_yc.loc[yield_categories & idx_vals, 'Report Category'] = report_category
    pds_supply_year_yield_pop_ref2_yc.loc[yield_categories & idx_vals, 'Report Category'] = report_category


In [68]:
pd.options.display.float_format = '{:20,.12f}'.format
pd.options.display.max_rows=1000
#pds_supply_year_yield_pop_ref2.loc[(['Australia','Brunei Darussalam','China','China, Hong Kong SAR','China, mainland','French Polynesia','Luxembourg','Republic of Korea','Samoa','Spain','Thailand'],'Aquatic Animals, Others'),'2014']#.groupby('Report Category').sum()['2020']
#pds_supply_year_yield_pop_ref2.loc[(['China, mainland'],'Aquatic Animals, Others'),slice(None)]
#kcal_per_gram.loc[(['Australia','Brunei Darussalam','China','China, Hong Kong SAR','China, mainland','French Polynesia','Luxembourg','Republic of Korea','Samoa','Spain','Thailand'],'Aquatic Animals, Others')]
population_medium.loc[['Australia','Brunei Darussalam','China, Taiwan Province of', 'China','China, Hong Kong SAR','French Polynesia','Luxembourg','Republic of Korea','Samoa','Spain','Thailand'],:]

KeyError: "Passing list-likes to .loc or [] with any missing labels is no longer supported. The following labels were missing: Index(['China, Taiwan Province of'], dtype='object', name='Country'). See https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike"

In [ ]:
test_PDS_supply = PDS_supply.copy()
aqua_cs = [
    'Australia',
    'Brunei Darussalam',
    'China',
    'China, Hong Kong SAR',
    'China, mainland',
    'French Polynesia',
    'Luxembourg',
    'Republic of Korea',
    'Samoa',
    'Spain',
    'Thailand']
#test_PDS_supply.loc[('China, mainland',slice(None)),:]
#test_PDS_supply = test_PDS_supply.loc[:,years].divide(other=kcal_per_gram, axis='index')
#test_PDS_supply = test_PDS_supply.multiply(other=population_medium, fill_value=0.0)
#test_PDS_supply *= 365
rpt = test_PDS_supply.loc[(slice(None) ,slice(None)),'2021':'2022'].join(food_waste_lookup['Yield category']).groupby(by=['Yield category','FAO commodity','Country']).sum()
rpt.loc[('Cereals',slice(None)),:]

In [70]:
# Supply_ADOPTION_DATE

# why are each of these are identical?
supply_results = {}

ref1_series = REF_supply_year_yield_pop_ref1_yc.groupby('Report Category').sum() / 10**6
ref1_series.loc['Meat'] += ref1_series.loc['Milk and egg']
ref1_series.drop(labels='Milk and egg', axis = 'rows', inplace=True)

ref2_series = REF_supply_year_yield_pop_ref2_yc.groupby('Report Category').sum() / 10**6
ref2_series.loc['Meat'] += ref2_series.loc['Milk and egg']
ref2_series.drop(labels='Milk and egg', axis = 'rows', inplace=True)
 
diet_series = diet_supply_year_yield_pop_ref2_yc.groupby('Report Category').sum() / 10**6
diet_series.loc['Meat'] += diet_series.loc['Milk and egg']
diet_series.drop(labels='Milk and egg', axis = 'rows', inplace=True)

waste_series = waste_supply_year_yield_pop_ref2_yc.groupby('Report Category').sum() / 10**6
waste_series.loc['Meat'] += waste_series.loc['Milk and egg']
waste_series.drop(labels='Milk and egg', axis = 'rows', inplace=True)

pds_series = pds_supply_year_yield_pop_ref2_yc.groupby('Report Category').sum() / 10**6
pds_series.loc['Meat'] += pds_series.loc['Milk and egg']
pds_series.drop(labels='Milk and egg', axis = 'rows', inplace=True)


ref1_ref2_diff = ref1_series.loc[:,years] - ref2_series.loc[:,years]
a = ref1_ref2_diff.loc['Grain', '2015':'2060'].sum() / 10**9
b = ref1_ref2_diff.loc['Meat', '2015':'2060'].sum() / 10**9
c = ref1_ref2_diff.loc['Fish and seafood', '2015':'2060'].sum() / 10**9
supply_results['Family Planning'] = (a,b,c)

ref2_diet_diff = ref2_series.loc[:,years] - diet_series.loc[:,years]
d = ref2_diet_diff.loc['Grain', '2015':'2060'].sum() / 10**9
e = ref2_diet_diff.loc['Meat', '2015':'2060'].sum() / 10**9
f = ref2_diet_diff.loc['Fish and seafood', '2015':'2060'].sum() / 10**9
supply_results['Plant-rich Diet only (75%)'] = (d,e,f)

ref2_waste_diff = ref2_series.loc[:,years] - waste_series.loc[:,years]
g = ref2_waste_diff.loc['Grain', '2015':'2060'].sum() / 10**9  
h = ref2_waste_diff.loc['Meat', '2015':'2060'].sum() / 10**9
i = ref2_waste_diff.loc['Fish and seafood', '2015':'2060'].sum() / 10**9
supply_results['Food waste only (75%)'] = (g,h,i)

ref2_pds_diff = ref2_series.loc[:,years] - pds_series.loc[:,years]
j = ref2_pds_diff.loc['Grain', '2015':'2060'].sum() / 10**9
k = ref2_pds_diff.loc['Meat', '2015':'2060'].sum() / 10**9
m = ref2_pds_diff.loc['Fish and seafood', '2015':'2060'].sum() / 10**9
supply_results['Integrated Food Solutions (Diet + Waste)'] = (j,k,m)

supply_results['Percent of yield reductions from diet soln'] = (100*d/j, 100*e/k, 100*f/m)
supply_results['Percent of yield reductions from waste soln'] = (100*(1- d/j),100*(1- e/k),100*(1- f/m))

report = pd.DataFrame(supply_results).transpose()
report.columns = ('crop-based reduction','livestock reduction','seafood reduction')
report


,crop-based reduction,livestock reduction,seafood reduction
Family Planning,16.249240416030,4.341611204583,0.587319656746
Plant-rich Diet only (75%),20.623251628448,11.079461924510,1.604274747781
Food waste only (75%),49.828374936975,6.131897673880,1.483911232373
Integrated Food Solutions (Diet + Waste),66.591495853941,15.899483037013,2.805152792896
Percent of yield reductions from diet soln,30.969797815749,69.684416145594,57.190280395564
Percent of yield reductions from waste soln,69.030202184251,30.315583854407,42.809719604436


In [71]:
# Yield_ADOPTION_DATE
yield_results = {}
ref1_series = REF_supply_year_yield_pop_ref1.sum()
ref2_series = REF_supply_year_yield_pop_ref2.sum()
diet_series = diet_supply_year_yield_pop_ref2.sum()
waste_series = waste_supply_year_yield_pop_ref2.sum()
pds_series = pds_supply_year_yield_pop_ref2.sum()

ref1_ref2_diff = ref1_series.loc[years] - ref2_series.loc[years]
a = ref1_ref2_diff.loc['2020':'2050'].sum() / 10**15
b = ref1_ref2_diff.loc['2015':'2060'].sum() / 10**15
yield_results['Family Planning'] = (a,b)

ref2_diet_diff = ref2_series.loc[years] - diet_series.loc[years]
c = ref2_diet_diff.loc['2020':'2050'].sum() / 10**15
d = ref2_diet_diff.loc['2015':'2060'].sum() / 10**15
yield_results['Plant-rich Diet only (75%)'] = (c,d)

ref2_waste_diff = ref2_series.loc[years] - waste_series.loc[years]
e = ref2_waste_diff.loc['2020':'2050'].sum() / 10**15
f = ref2_waste_diff.loc['2015':'2060'].sum() / 10**15
yield_results['Food waste only (75%)'] = (e,f)

ref2_pds_diff = ref2_series.loc[years] - pds_series.loc[years]
g = ref2_pds_diff.loc['2020':'2050'].sum() / 10**15
h = ref2_pds_diff.loc['2015':'2060'].sum() / 10**15
yield_results['Integrated Food Solutions (Diet + Waste)'] = (g,h)

yield_results['Percent of yield reductions from diet soln'] = (100*c/g, 100*d/h)
yield_results['Percent of yield reductions from waste soln'] = (100*(1- c/g),100*(1- d/h))

report = pd.DataFrame(yield_results).transpose()
report.columns = ('2020-50 cumulative reduction','2015-60 cumulative reduction')
report



,2020-50 cumulative reduction,2015-60 cumulative reduction
Family Planning,10.427293454034,21.178171277359
Plant-rich Diet only (75%),18.384927765527,33.306988300740
Food waste only (75%),35.564428755648,57.444183843229
Integrated Food Solutions (Diet + Waste),51.333790010246,85.296131683850
Percent of yield reductions from diet soln,35.814475731984,39.048650440787
Percent of yield reductions from waste soln,64.185524268016,60.951349559213


In [75]:
diet_series
#pds_supply_year_yield_pop_ref2_yc.groupby('Yield category').sum()
#PDS_supply.loc[(slice(None),'Aquatic Animals, Others'),:]


2014                  11,587,948,709.122352600098
2015                  11,684,747,753.777906417847
2016                  11,843,676,835.958019256592
2017                  12,000,012,473.394548416138
2018                  12,264,564,431.508968353271
2019                  12,367,705,871.530282974243
2020                  12,464,897,939.373140335083
2021                  12,545,579,272.016208648682
2022                  12,619,235,524.869361877441
2023                  12,686,072,275.928535461426
2024                  12,746,214,751.636228561401
2025                  12,799,752,915.224052429199
2026                  12,846,755,148.564968109131
2027                  12,887,273,613.632560729980
2028                  12,921,322,279.854499816895
2029                  12,948,893,495.420490264893
2030                  12,969,966,362.126806259155
2031                  12,962,322,064.552276611328
2032                  12,949,338,106.055557250977
2033                  12,931,158,060.880607604980


In [73]:
# Emissions_ADOPTION_DATE

emissions_results = {}
ref1_series = REF1_emissions.sum()
ref2_series = REF2_emissions.sum()
diet_series = diet_annual_emissions.sum()
waste_series = waste_emissions.sum()
pds_series = pds_annual_emissions.sum()

ref1_ref2_diff = ref1_series.loc[years] - ref2_series.loc[years]
a = ref1_ref2_diff.loc['2020':'2050'].sum() / 10**9
b = ref1_ref2_diff.loc['2015':'2060'].sum() / 10**9
emissions_results['Family Planning'] = (a,b)

ref2_diet_diff = ref2_series.loc[years] - diet_series.loc[years]
c = ref2_diet_diff.loc['2020':'2050'].sum() / 10**9
d = ref2_diet_diff.loc['2015':'2060'].sum() / 10**9
emissions_results['Plant-rich Diet only (75%)'] = (c,d)

ref2_waste_diff = ref2_series.loc[years] - waste_series.loc[years]
e = ref2_waste_diff.loc['2020':'2050'].sum() / 10**9
f = ref2_waste_diff.loc['2015':'2060'].sum() / 10**9
emissions_results['Food waste only (75%)'] = (e,f)

ref2_pds_diff = ref2_series.loc[years] - pds_series.loc[years]
g = ref2_pds_diff.loc['2020':'2050'].sum() / 10**9
h = ref2_pds_diff.loc['2015':'2060'].sum() / 10**9
emissions_results['Integrated Food Solutions (Diet + Waste)'] = (g,h)

emissions_results['Percent of yield reductions from diet soln'] = (100*c/g, 100*d/h)
emissions_results['Percent of yield reductions from waste soln'] = (100*(1- c/g),100*(1- d/h))

report = pd.DataFrame(emissions_results).transpose()
report.columns = ('2020-50 cumulative reduction','2015-60 cumulative reduction')
report

,2020-50 cumulative reduction,2015-60 cumulative reduction
Family Planning,21.595880936278,44.327115913126
Plant-rich Diet only (75%),108.851472408473,183.747466996784
Food waste only (75%),48.026016177801,78.483812814672
Integrated Food Solutions (Diet + Waste),144.667315892255,239.362794759077
Percent of yield reductions from diet soln,75.242615608866,76.765258018369
Percent of yield reductions from waste soln,24.757384391134,23.234741981631


In [74]:
#The datasets calculated below match the tables on their respective worksheet tabs in the s/sht.
# Note that the s/sht tables are not updated properly. S/sht formulas need to be copied down and recalculated.

#REF_country_supply
REF_supply_year_yield_pop_ref1.groupby('Country').sum()/10**6
REF_supply_year_yield_pop_ref2.groupby('Country').sum()/10**6

#PDS_country_supply
diet_supply_year_yield_pop_ref2.groupby('Country').sum()/10**6
waste_supply_year_yield_pop_ref2.groupby('Country').sum()/10**6
pds_supply_year_yield_pop_ref2.groupby('Country').sum()/10**6

# REF_commodity_supply
REF_supply_year_yield_pop_ref1.groupby('FAO commodity').sum()/10**6
REF_supply_year_yield_pop_ref2.groupby('FAO commodity').sum()/10**6

diet_supply_year_yield_pop_ref2.groupby('FAO commodity').sum()/10**6
waste_supply_year_yield_pop_ref2.groupby('FAO commodity').sum()/10**6
pds_supply_year_yield_pop_ref2.groupby('FAO commodity').sum()/10**6


,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,...,2091,2092,2093,2094,2095,2096,2097,2098,2099,2100
FAO commodity,,,,,,,,,,,,,,,,,,,,,
Apples and products,"88,964,019.694372192025","87,807,542.808678269386","92,301,062.720835775137","95,625,292.088377177715","98,978,052.734830424190","98,708,552.709313660860","98,480,119.413026705384","98,085,121.980563759804","97,635,982.543744802475","97,137,620.513799577951",...,"59,138,827.103682331741","58,790,717.981379926205","58,442,020.779137201607","58,092,512.989034906030","57,741,935.830933988094","57,389,992.272494569421","57,036,359.672246284783","56,680,683.757155679166","56,322,565.943582154810","55,961,581.455152414739"
"Aquatic Animals, Others","1,131,836.245080685010","1,137,850.273684184300","1,149,388.822477878304","1,055,462.016219315585","1,060,435.154493320035","1,028,432.133519675350","1,008,048.013645268627","985,922.635567666846","963,088.598726284690","939,642.895309992135",...,"172,566.911510922160","170,896.925596848334","169,232.173046043579","167,569.945969737193","165,907.219696490269","164,240.808367190155","162,567.377872040204","160,883.501922345196","159,185.625127945648","157,470.073155034945"
Aquatic Plants,"1,653,594.819670117926","1,802,591.973551212810","2,022,487.104520883877","2,600,860.784517111722","2,604,084.394120083191","2,541,134.424858177081","2,477,999.304218899924","2,412,057.744056873955","2,345,507.819113424979","2,278,468.627499471419",...,"326,641.103432546195","322,499.124432187644","318,386.433541571721","314,313.758823987446","310,292.569220586796","306,332.604902360355","302,443.860049101524","298,634.032966163475","294,910.557782167278","291,279.149631637265"
Bananas,"129,617,626.882654979825","137,497,515.274934381247","131,686,021.633330836892","133,461,278.563705131412","134,264,055.412002056837","135,547,847.691929668188","136,887,396.114233314991","138,103,291.545158952475","139,272,394.628709226847","140,394,216.394370198250",...,"168,249,903.699007213116","168,475,412.024843126535","168,690,105.704728037119","168,894,365.454089999199","169,088,494.991116523743","169,272,754.624470651150","169,447,323.016810476780","169,612,295.437977433205","169,767,688.815597087145","169,913,434.112690985203"
Barley and products,"8,813,372.962349347770","9,008,331.561867704615","8,323,637.922436702996","8,486,125.774194322526","9,306,298.322232397273","9,368,520.606973737478","9,428,377.819223733619","9,482,363.585811989382","9,533,224.272641040385","9,581,202.516092186794",...,"11,107,093.201387550682","11,105,927.101187439635","11,103,500.219737458974","11,099,811.646477222443","11,094,855.419242542237","11,088,626.283290890977","11,081,117.648194395006","11,072,320.811481021345","11,062,225.999835496768","11,050,822.141116501763"
Beans,"20,573,608.658560540527","20,597,192.487326931208","20,861,858.620720617473","22,261,485.378062024713","22,462,236.525990553200","23,596,920.427485510707","24,752,375.482852887362","25,909,521.709171071649","27,077,864.149059802294","28,257,380.399220351130",...,"71,626,637.438037291169","71,745,438.594659954309","71,858,244.691088810563","71,965,236.786991998553","72,066,584.499108880758","72,162,446.820331305265","72,252,963.820967674255","72,338,260.197825685143","72,418,434.928449347615","72,493,570.613776728511"
Beer,"230,040,838.080907493830","229,788,042.042390853167","219,845,424.212743729353","217,291,264.335488617420","207,646,886.131983280182","203,298,536.859941005707","199,079,595.489753752947","194,543,934.079389274120","189,925,867.572622299194","185,236,812.946353763342",...,"47,054,112.126842133701","47,020,330.475057832897","46,985,536.262526474893","46,949,635.208248205483","46,912,517.863548383117","46,874,061.357634492218","46,834,132.839622683823","46,792,589.207324765623","46,749,267.687608726323","46,703,996.264238886535"
"Beverages, Alcoholic","26,110,585.414515838027","25,120,144.764167949557","25,294,221.480847939849","25,147,654.090033974499","26,518,557.297136962414","25,9